In [ ]:
!pip install influxdb-client

In [24]:
import pandas as pd
import seaborn as sns

# read data from csv
df = pd.read_csv('bikeSharing.csv')

# cast datetime column from string to dteday format
df['datetime'] = pd.to_datetime(df['dteday'])

# check the results
df.info()

In [8]:
from datetime import datetime

from influxdb_client import InfluxDBClient, Point, WritePrecision
from influxdb_client.client.write_api import SYNCHRONOUS

# You can generate an API token from the "API Tokens Tab" in the UI
token = "sda-token"
org = "sda"
bucket = "training"

In [32]:
# Create a line protocol data sequence by iterating over the data
sequence = []
for index, row in df.iterrows():
    
    pointWe = (Point("weatherInfo")
            .tag("area", "Washington DC")  
            .field("temp", row['temp'])
            .field("atemp", row['atemp'])
            .field("humidity", row['hum'])
            .field("windspeed", row['windspeed'])
            .field("weather", row['weathersit'])
            .time(row['datetime']))
    sequence+= [pointWe.to_line_protocol()]
    
    pointBsc = (Point("bikesharingInfo")
            .tag("area", "Washington DC") 
            .tag("user", "casual")
            .field("cnt", row['casual'])
            .time(row['datetime']))
    sequence+= [pointBsc.to_line_protocol()]
    
    pointBsr = (Point("bikesharingInfo")
            .tag("area", "Washington DC") 
            .tag("user", "registered")
            .field("cnt", row['registered'])
            .time(row['datetime']))
    sequence+= [pointBsr.to_line_protocol()]

In [33]:
# write data to influxdb
with InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    write_api.write(bucket, org, sequence)